# Functions

In Julia, a function is an object that maps a tuple of argument values to a return value. In the following example we define function that

In [1]:
function f(x,y)
    x * y
end

f (generic function with 1 method)

This function accepts two arguments `x` and `y` and returns the value of the last expression evaluated, which is `x * y`.

In [2]:
[f(2, 3), f(2, -3)]

2-element Array{Int64,1}:
  6
 -6

Sometimes it is useful to return something other than the last expression.  For such a case there is an `return` keyword:

In [3]:
function g(x,y)
    val = x * y
    if val < 0
        return -val
    else
        return val
    end
end

g (generic function with 1 method)

This function accepts two arguments `x` and `y` and computes `val = x * y`. Then if `val` is less than zero, it returns` -val`, otherwise it returns `val`.

In [4]:
[g(2, 3), g(2, -3)]

2-element Array{Int64,1}:
 6
 6

The traditional function declaration syntax demonstrated above is equivalent to the following compact form, which is very common in Julia:

In [5]:
f(x,y) = x * y

f (generic function with 1 method)

### Optional and keyword arguments

Other very useful things are optional and keyword arguments, which can be added in a very easy way

In [6]:
function f_hello(x, y, a = 0; sayhello = false)
    sayhello && println("Hello everyone 👋" )
    x * y + a
end

f_hello (generic function with 2 methods)

This function accepts two arguments `x` and `y`, one optional argument `a` and one keyword argument `sayhello`. If the function is called only with mandatory arguments, then it returns `x * y + 0`

In [7]:
f_hello(2,3)

6

The change of the optional argument `a` will change the output value to `x * y + a`

In [8]:
f_hello(2,3,2)

8

Since `f_hello` is a function with good manners (as opposed to `f`), it says hello if the keyword argument `sayhello` is true

In [9]:
f_hello(2,3; sayhello = true)

Hello everyone 👋


6

### Anonymous functions

It is also common to use anonymous functions, ie functions without specified name. Such a function can be defined in almost the same way as a normal function:

In [10]:
h1 = function (x)
    x^2 + 2x - 1
end

h2 = x ->  x^2 + 2x - 1

#6 (generic function with 1 method)

 Those two function declarations create functions with automatically generated names. Then variables `h1` and `h2` only refers to these functions. The primary use for anonymous functions is passing them to functions which take other functions as arguments. A classic example is `map`, which applies a function to each value of an array and returns a new array containing the resulting values:

In [11]:
map(x -> x^2 + 2x - 1, [1,3,-1])

3-element Array{Int64,1}:
  2
 14
 -2

For more complicated functions, the `do` blocks can be used

In [12]:
map([1,3,-1]) do x
    x^2 + 2x - 1
end

3-element Array{Int64,1}:
  2
 14
 -2

# Types, methods and multiple-dispatch

So far we did not mention any types. The default behavior in Julia when types are omitted is to allow values to be of any type. Thus, one can write many useful Julia functions without ever explicitly using types. When additional expressiveness is needed, however, it is easy to gradually introduce explicit type annotations into previously "untyped" code.

 In Julia, functions consist of multiple methods. The choice of which method to execute when a function is applied is called dispatch. Julia allows the dispatch process to choose which of a function's methods to call based on
* the number of arguments given
* types of all of the function's arguments.

Using all of a function's arguments to choose which method should be invoked is known as **multiple dispatch**.

Until now, we have defined only functions with a single method having unconstrained argument types.

In [13]:
f(x, y) = x * y

f (generic function with 1 method)

We can easily check which methods are defined for this function using the `methods` function

In [14]:
methods(f)

# 1 method for generic function "f":
[1] f(x, y) in Main at In[13]:1

Each function can be easily extended by new methods

In [15]:
f(x, y, z) = x * y * z
f(x, y, z, q) = x * y * z * q
f(x...) = reduce(*, x)

methods(f)

# 4 methods for generic function "f":
[1] f(x, y) in Main at In[13]:1
[2] f(x, y, z) in Main at In[15]:1
[3] f(x, y, z, q) in Main at In[15]:2
[4] f(x...) in Main at In[15]:3

Since we did not specify what types of arguments are allowed, function `f` will work for all types

In [16]:
[
    f(2, 3),
    f(2.0, 3),
    f(2, 3.0),
    f("a", "b")
]

4-element Array{Any,1}:
 6
 6.0
 6.0
  "ab"

However, some combinations of arguments will result in an error

In [17]:
f(:a, :b)

MethodError: MethodError: no method matching *(::Symbol, ::Symbol)
Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:529

 When using types we can be extremely conservative and we can set a specific type for each function argument

In [18]:
foo(x::Int64, y::Int64) = x * y

foo (generic function with 1 method)

This function definition applies only to calls where `x` and `y` are both values of type Int64:

In [19]:
foo(2,3)

6

Applying it to any other types of arguments will result in a `MethodError`:

In [20]:
foo(2.0,3)

MethodError: MethodError: no method matching foo(::Float64, ::Int64)
Closest candidates are:
  foo(!Matched::Int64, ::Int64) at In[18]:1

It is better to use abstract types like `Number` or` Real` instead of concrete types like `Float64`,` Float32`, `Int64` ... .  To find an super type for a specific type, we can use  `supertype` function

In [21]:
supertype(Int64)

Signed

or we can create a simple recursive function that prints the entire tree of supertypes for a given type

In [22]:
function supertypes_tree(::Type{T}, k::Int = 0) where {T <: Any}
    T === Any && return
    col = isabstracttype(T) ? :blue : :green
    printstyled(repeat("   ", k)..., T, "\n"; bold = true, color = col)
    supertypes_tree(supertype(T), k + 1)
    return
end

supertypes_tree(Int64)

Int64
   Signed
      Integer
         Real
            Number


 All abstract types are printed in blue and all concrete types are printed in green. There is also `subtypes` function, which returns all subtypes for a given type.

In [23]:
subtypes(Number)

2-element Array{Any,1}:
 Complex
 Real

As with supertypes, we can create a simple recursive function that prints the entire tree of subtypes for a given type.

In [24]:
function subtypes_tree(::Type{T}, k::Int = 0) where {T <: Any}
    col = isabstracttype(T) ? :blue : :green
    printstyled(repeat("   ", k)..., T; bold = true, color = col)
    println()
    subtypes_tree.(subtypes(T), k + 1)
    return
end


subtypes_tree(Number)

Number
   Complex
   Real
      AbstractFloat
         BigFloat
         Float16
         Float32
         Float64
      AbstractIrrational
         Irrational
      Integer
         Bool
         Signed
            BigInt
            Int128
            Int16
            Int32
            Int64
            Int8
         Unsigned
            UInt128
            UInt16
            UInt32
            UInt64
            UInt8
      Rational


From the tree of all subtypes of the abstract type "Number," we see the whole structure of numerical types in Julia. So if we really want to specify the argument types of a function, we should use some abstract type, such as `Real`

In [25]:
foo(x::Real, y::Real) = x * y

foo (generic function with 2 methods)

This function definition applies to calls where `x` and `y` are both values of any subtype of `Real`.

In [26]:
Real[
    foo(2.0, 3)
    foo(2.0, 3.0)
    foo(Int32(2), Int16(3.0))
    foo(Int32(2), Float32(3.0))
]

4-element Array{Real,1}:
 6.0
 6.0
 6
 6.0f0

Now we can check again how many methods are defined for `foo`

In [27]:
methods(foo)

# 2 methods for generic function "foo":
[1] foo(x::Int64, y::Int64) in Main at In[18]:1
[2] foo(x::Real, y::Real) in Main at In[25]:1

### Method Ambiguities

It is possible to define a set of function methods such that there is no unique most specific method applicable to some combinations of arguments:

In [28]:
goo(x::Float64, y) = x * y
goo(x, y::Float64) = x + y

goo (generic function with 2 methods)

 Here, the `goo` function has two methods. The first method applies if the first argument is of type `Float64`.

In [29]:
goo(2.0, 3)

6.0

The second method applies if the second argument is of type `Float64`.

In [30]:
goo(2, 3.0)

5.0

The case, where both arguments are of type `Float64` can be handled by both methods. The problem is that neither method is more specific than the other. In such cases, Julia raises a `MethodError` rather than arbitrarily picking a method.

In [31]:
goo(2.0, 3.0)

MethodError: MethodError: goo(::Float64, ::Float64) is ambiguous. Candidates:
  goo(x::Float64, y) in Main at In[28]:1
  goo(x, y::Float64) in Main at In[28]:2
Possible fix, define
  goo(::Float64, ::Float64)

 We can avoid method ambiguities by specifying an appropriate method for the intersection case:

In [32]:
goo(x::Float64, y::Float64) = x - y
goo(2.0, 3.0)

-1.0

If we can check again how many methods are defined for `goo`, there will be three methods

In [33]:
methods(goo)

# 3 methods for generic function "goo":
[1] goo(x::Float64, y::Float64) in Main at In[32]:1
[2] goo(x::Float64, y) in Main at In[28]:1
[3] goo(x, y::Float64) in Main at In[28]:2

### Composite types

Composite types are called records, structs, or objects in various languages. A composite type is a collection of named fields, an instance of which can be treated as a single value. In many languages, composite types are the only kind of user-definable type, and they are by far the most commonly used user-defined type in Julia as well.

In [34]:
abstract type Food{T<:Real} end
abstract type Fruit{T<:Real} <: Food{T} end
abstract type Vegetable{T<:Real} <: Food{T} end

struct Apple{T<:Real} <: Fruit{T}  weight::T end
struct Orange{T<:Real} <: Fruit{T} weight::T end
struct Banana{T<:Real} <: Fruit{T} weight::T end

struct Cucumber{T<:Real} <: Vegetable{T} weight::T end
struct Carrot{T<:Real} <: Vegetable{T} weight::T end
struct Lettuce{T<:Real} <: Vegetable{T} weight::T end

Using the `subtypes_tree` function, we can easily check the type hierarchy

In [35]:
subtypes_tree(Food)

Food
   Fruit
      Apple
      Banana
      Orange
   Vegetable
      Carrot
      Cucumber
      Lettuce


In Julia, it is not possible to set mandatory fields for all subtypes of a given abstract type. For example, each food subtype should have a specified color. However, we can easily define general properties using multiple-dispatch

In [36]:
color(::Type{<:Apple}) = "red"
color(::Type{<:Orange}) = "orange"
color(::Type{<:Banana}) = "yellow"
color(::Type{<:Cucumber}) = "green"
color(::Type{<:Carrot}) = "orange"
color(::Type{<:Lettuce}) = "green"

color (generic function with 6 methods)

However, these methods can be applied only to the type itself.

In [37]:
[color(Apple), color(Lettuce)]

2-element Array{String,1}:
 "red"
 "green"

To apply `color` function to a specific instance of any `Food` subtype, we must do the following

In [38]:
a = Apple(123)
color(typeof(a))

"red"

However, it can be also done in a more elegant way using a new method.

In [39]:
color(::T) where {T<:Food} = color(T)

[color(Apple), color(Apple(123))]

2-element Array{String,1}:
 "red"
 "red"

Now we can define two other functions:
* `weight` function return the rounded weight of given food
* `description` function prints some basic information about a given food

In [40]:
weight(x::Food) = ceil(Int64, x.weight)
description(x::T) where {T <: Food} =
    println("$(supertype(T).name): $(T.name), color: $(color(x)), weight: $(weight(x))g")

fruits = [
    Apple(150),
    Orange(235.4),
    Banana(186.6),
    Cucumber(246.1),
    Carrot(120),
    Lettuce(169)
]

description.(fruits);

Fruit: Apple, color: red, weight: 150g
Fruit: Orange, color: orange, weight: 236g
Fruit: Banana, color: yellow, weight: 187g
Vegetable: Cucumber, color: green, weight: 247g
Vegetable: Carrot, color: orange, weight: 120g
Vegetable: Lettuce, color: green, weight: 169g


It is very useful to know which functions are defined for a particular type. We can use the `methodswith` function to get such information.

In [41]:
methodswith(Apple; supertypes = true)

[1] color(::T) where T<:Food in Main at In[39]:1
[2] description(x::T) where T<:Food in Main at In[40]:2
[3] weight(x::Food) in Main at In[40]:1

# Complex example: Temperatures ([original source](https://medium.com/@Jernfrost/defining-custom-units-in-julia-and-python-513c34a4c971))

In this example, we will show how to deal with temperatures in different units (*Celsius*, *Kelvin*, *Fahrenheit*). We have following goals:

1. define types that represent temperature units
2. define functions for conversion between temperature types
3. define basic arithmetic operations for temperature types
    * `Temperature + Temperature`
    * `Temperature - Temperature`
    * `number * Temperature`
    * `Temperature / number`

First, we define the abstract type `Temperature`. All of the above functions can be implemented without the use of the `Temperature` type, but it will be much more complicated.

In [42]:
abstract type Temperature end

We can now define a `Celsius` structure that will represent the temperature in degree *Celsius*.

In [43]:
struct Celsius <: Temperature
    value::Float64

    function Celsius(x::Real)
        x < -273.15 && throw(ArgumentError("input temperature smaller than absolute zero"))
        return new(x)
    end
end

The previous definition is valid for any real number greater than or equal to *-273.15*, which is absolute zero in degrees Celsius.

In [44]:
(Celsius(-273.15), Celsius(0), Celsius(100))

(Celsius(-273.15), Celsius(0.0), Celsius(100.0))

Since Julia supports multiple-dispatch, we can easily extend existing functions to support newly defined types. For example, we can extend the `show` function from the` Base` module to change the way `Celsius` type is printed in REPL.

In [45]:
Base.show(io::IO, t::Celsius) = print(io, t.value, "°C")

Using the same values as above, we get the following output

In [46]:
(Celsius(-273.15), Celsius(0), Celsius(100))

(-273.15°C, 0.0°C, 100.0°C)

In  the same way, we can easily define another temperature scale.

In [47]:
struct Kelvin <: Temperature
    value::Float64

    function Kelvin(x::Real)
        x < 0 && throw(ArgumentError("input temperature smaller than absolute zero"))
        return new(x)
    end
end

Base.show(io::IO, t::Kelvin) = print(io, t.value, "K")

## Conversion

We are now able to express temperatures in two different units, but we are not able to convert from one unit to another. In order to convert between units, we need to create a conversion function.

In [48]:
Celsius2kelvin(t::Celsius) = Kelvin(t.value + 273.15)
Kelvin2Celsius(t::Kelvin) = Celsius(t.value - 273.15)

[
    Celsius2kelvin(Celsius(0)),
    Kelvin2Celsius(Kelvin(0))
]

2-element Array{Temperature,1}:
 273.15K
 -273.15°C

However, the better way is to extend the `convert` function from the` Base` module and combine it with outer constructors for temperature types. In the first step, we define conversion rules using outer constructors

In [49]:
Kelvin(t::Celsius) = Kelvin(t.value + 273.15)
Celsius(t::Kelvin) = Celsius(t.value - 273.15)

Celsius

In the second step, we define `convert` function for any subtype of the abstract type `Temperature`.

In [50]:
Base.convert(::Type{T}, t::T) where {T<:Temperature} = t
Base.convert(::Type{T}, t::Temperature) where {T<:Temperature} = T(t)

The first method is only a temperature identity. The second method is an auxiliary function that passes the given temperature `t` to the constructor of the given type of temperature `T`. Using the same example as for the `temp_convert` function above results in

In [51]:
[
    convert(Celsius, Celsius(0)),
    convert(Kelvin, Kelvin(0)),
    convert(Kelvin, Celsius(0)),
    convert(Celsius, Kelvin(0))
]

4-element Array{Temperature,1}:
 0.0°C
 0.0K
 273.15K
 -273.15°C

### Basic arithmetic operations

Before defining any arithmetic operation, we must define the right way to deal with cases where we have to deal with temperatures in different temperature scales. To do that, we have to define  `promote_rule` for our types

In [52]:
Base.promote_rule(::Type{Kelvin}, ::Type{Celsius}) = Kelvin

[
    promote_type(Celsius, Kelvin)
    promote(Celsius(-273.15), Kelvin(0))
]

2-element Array{Any,1}:
 Kelvin
 (0.0K, 0.0K)

We can now define basic arithmetic operations in two easy steps as can be seen in the following code

In [53]:
import Base: +, -, *, /

+(x::Temperature, y::Temperature) = +(promote(x,y)...)
+(x::T, y::T) where {T<:Temperature} = T(x.value + y.value)

-(x::Temperature, y::Temperature) = -(promote(x,y)...)
-(x::T, y::T) where {T<:Temperature} = T(x.value - y.value)

- (generic function with 176 methods)

Now we are able to add and subtract temperatures in different temperature scales

In [54]:
[
    Celsius(-273.15) + Kelvin(0),
    Kelvin(0) + Celsius(-273.15),
    Celsius(-273.15) - Kelvin(0),
    Kelvin(0) - Celsius(-273.15)
]

4-element Array{Kelvin,1}:
 0.0K
 0.0K
 0.0K
 0.0K

We can also define the multiplication of temperature by a number and rounding function

In [55]:
*(x::Number, y::T) where {T <: Temperature} = T(x * y.value)
*(x::T, y::Number) where {T <: Temperature} = T(y * x.value)

Base.round(t::T, args...; kwargs...) where {T<:Temperature} = T(round(t.value, args...; kwargs...))

In Julia, it is possible to apply given function `f(x)` to each element of an array `A` to yield a new array via `f.(A)`. We can use this syntax to obtain a random temperature vector in degrees Kelvin as follows

In [56]:
temps_K = Kelvin.(273.15 .+ 20 .* rand(10))

10-element Array{Kelvin,1}:
 288.1982985661692K
 281.3070076094924K
 277.1532737013006K
 291.5523107033456K
 283.87707492275484K
 289.28941044682995K
 284.04591738879674K
 284.5776106099723K
 284.21514378740085K
 282.1347598213607K

 In the same way, we can convert this vector to degrees Celsius and round it to two digits

In [57]:
temps_C = round.(Celsius.(temps_K); digits = 2)

10-element Array{Celsius,1}:
 15.05°C
 8.16°C
 4.0°C
 18.4°C
 10.73°C
 16.14°C
 10.9°C
 11.43°C
 11.07°C
 8.98°C

Finally, we can compute for example the sum of this vector

In [58]:
sum(temps_C)

114.86°C

### Adding new temperature scale

To add a new temperature scale, we have to:
* define new type
* extend `Base.show` (otpional)
* define outer constructors for `Kelvin` and `Celsius`
* define promote rules for `Kelvin` and `Celsius`

In [59]:
struct Fahrenheit <: Temperature
    value::Float64

    function Fahrenheit(x::Real)
        x < -459.67 && throw(ArgumentError("input temperature smaller than absolute zero"))
        return new(x)
    end
end

Base.show(io::IO, t::Fahrenheit) = print(io, t.value, "°F")

Celsius(t::Fahrenheit) = Celsius((t.value - 32)*5/9)
Fahrenheit(t::Celsius) = Fahrenheit(t.value*9/5 + 32)
Kelvin(t::Fahrenheit) = Kelvin(Celsius(t))
Fahrenheit(t::Kelvin) = Fahrenheit(Celsius(t))

Base.promote_rule(::Type{Fahrenheit}, ::Type{Celsius}) = Celsius
Base.promote_rule(::Type{Fahrenheit}, ::Type{Kelvin}) = Kelvin

Now all the functions defined for `Kelvin` and `Celsius` will work for `Fahrenheit` as well

In [60]:
Temperature[
    Fahrenheit(Celsius(0)),
    Celsius(Fahrenheit(32)),
    Fahrenheit(Kelvin(0)),
    Kelvin(Fahrenheit(-459.67)),
]

Fahrenheit(32) + Celsius(0) - Kelvin(273.15)

0.0K

To obtain even more user-friendly behavior, we can define constants representing 1 degree in each temperature scale

In [61]:
const °C = Celsius(1)
const K = Kelvin(1)
const °F = Fahrenheit(1)

1.0°F

 With these constants and the fact, that `*` operator can be omitted in some cases, we can work with temperatures as follows

In [66]:
2°C + 4K - 2°F

22.666666666666686K

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*